In [3]:
import os, sys
currentdir = os.path.dirname(os.path.realpath("Tutorial_Rocks_Inversion.ipynb"))
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

In [4]:
# MODULES
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from modules import sequences, minerals, fluids
from modules.sequences import SedimentaryBasin
from modules import geophysics
from pandas.plotting import scatter_matrix
import seaborn as sn
from scipy.optimize import nnls, minimize
import scipy.optimize as opt

In [5]:
# Test feldspathic sandstone generation within SedimentaryBasin class
data = SedimentaryBasin()
data_sandstone = data.create_sandstone(thickness=10, keyword="feldspathic")
print(data_sandstone[0][-1][-1])

data = sequences.DataProcessing(dataset=data_sandstone)
density_bulk = data.extract_densities(type="rock", keyword="bulk")
vP = data.extract_seismic_velocities(type="rock", keyword="vP")
vS = data.extract_seismic_velocities(type="rock", keyword="vS")
gr = data.extract_gamma_ray(type="rock")
pe = data.extract_photoelectricity(type="rock")

d = []
for i in range(len(data_sandstone)):
    d.append([density_bulk[i]*1000, vP[i], vS[i], gr[i], pe[i]])
    print("Sample", i+1, ":", d[i])
d = np.array(d)

[0.5198, 0.0761, 0.4041]
Sample 1 : [2160.0, 2861.83, 1598.88, 15.646, 1.855]
Sample 2 : [2176.0, 2841.99, 1602.2, 13.171, 1.892]
Sample 3 : [2142.0, 2811.01, 1564.54, 15.417, 1.827]
Sample 4 : [2216.0, 2915.14, 1656.87, 8.604, 2.005]
Sample 5 : [2182.0, 2880.84, 1623.84, 8.261, 1.856]
Sample 6 : [2152.0, 2846.66, 1591.8, 4.336, 1.706]
Sample 7 : [2226.0, 2904.36, 1670.32, 7.105, 1.881]
Sample 8 : [2222.0, 2865.59, 1655.0, 4.521, 1.819]
Sample 9 : [2224.0, 2878.87, 1673.32, 1.139, 1.637]
Sample 10 : [2193.0, 2856.81, 1629.85, 1.418, 1.745]


In [6]:
data_qz = minerals.oxides.quartz("")
data_kfs = minerals.feldspars().alkalifeldspar(keyword="Kfs")
data_pl = minerals.feldspars().plagioclase(keyword="Pl")
water = fluids.Water.water("")

properties_rho = [data_qz[2], data_kfs[2], data_pl[2], water[2]]
properties_vP = [data_qz[4][0], data_kfs[4][0], data_pl[4][0], water[4][0]]
properties_vS = [data_qz[4][1], data_kfs[4][1], data_pl[4][1], water[4][1]]
properties_GR = [data_qz[5][0], data_kfs[5][0], data_pl[5][0], water[5][0]]
properties_PE = [data_qz[5][1], data_kfs[5][1], data_pl[5][1], water[5][1]]

properties_rho_s = [data_qz[2], data_kfs[2], data_pl[2]]
properties_vP_s = [data_qz[4][0], data_kfs[4][0], data_pl[4][0]]
properties_vS_s = [data_qz[4][1], data_kfs[4][1], data_pl[4][1]]
properties_GR_s = [data_qz[5][0], data_kfs[5][0], data_pl[5][0]]
properties_PE_s = [data_qz[5][1], data_kfs[5][1], data_pl[5][1]]

G = np.array([properties_rho, properties_vP, properties_vS, properties_GR, properties_PE])
G_s = np.array([properties_rho_s, properties_vP_s, properties_vS_s, properties_GR_s, properties_PE_s])
#print(G)

G_T = np.transpose(G)
G_TG = G_T@G
GTG_inv = np.linalg.inv(G_TG)
m = GTG_inv@G_T@d[0]
#print(m)

G_T_s = np.transpose(G_s)
G_TG_s = G_T_s@G_s
GTG_inv_s = np.linalg.inv(G_TG_s)
m_s = GTG_inv_s@G_T_s@d[0]
#print(m_s)

In [7]:
m = np.linalg.lstsq(a=G, b=d[0], rcond=None)[0]
print("Model estimation:", m, m.sum())
print("Check:")
print("Density:", round(np.dot(m,properties_rho), 2))
print("vP:", round(np.dot(m,properties_vP), 2))
print("vS:", round(np.dot(m,properties_vS), 2))
print("GR:", round(np.dot(m,properties_GR), 2))
print("PE:", round(np.dot(m,properties_PE), 2))
print("Reality:")
print(d[0])

Model estimation: [ 0.91396241  0.08812797 -0.49839856  0.82180284] 1.3254946569479116
Check:
Density: 2160.0
vP: 2861.83
vS: 1598.88
GR: 15.65
PE: 1.18
Reality:
[2.16000e+03 2.86183e+03 1.59888e+03 1.56460e+01 1.85500e+00]


In [6]:
m_s = np.linalg.lstsq(a=G_s, b=d[0], rcond=None)[0]
print("Model estimation:", m_s, m_s.sum())
print("Check:")
print("Density:", round(np.dot(m_s, properties_rho_s), 2))
print("vP:", round(np.dot(m_s, properties_vP_s), 2))
print("vS:", round(np.dot(m_s, properties_vS_s), 2))
print("GR:", round(np.dot(m_s, properties_GR_s), 2))
print("PE:", round(np.dot(m_s, properties_PE_s), 2))
print("Reality:")
print(d[0])

Model estimation: [ 0.99134964 -0.17949695 -0.19378489] 0.6180677927579908
Check:
Density: 1645.21
vP: 2531.53
vS: 2155.14
GR: -32.26
PE: 0.8
Reality:
[2.12100e+03 2.75724e+03 1.52737e+03 5.85800e+00 1.73800e+00]


In [7]:
A = G
b = d[0]

m0, rnorm = nnls(A, b)
print("Model estimation:", m0, m0.sum())
print("Check:")
print("Density:", round(np.dot(m0,properties_rho), 2))
print("vP:", round(np.dot(m0,properties_vP), 2))
print("vS:", round(np.dot(m0,properties_vS), 2))
print("GR:", round(np.dot(m0,properties_GR), 2))
print("PE:", round(np.dot(m0,properties_PE), 2))
print("Reality:")
print(d[0])

Model estimation: [0.33883775 0.         0.         0.77234775] 1.1111855001794555
Check:
Density: 1669.79
vP: 3068.61
vS: 1381.07
GR: 0.0
PE: 0.84
Reality:
[2.12100e+03 2.75724e+03 1.52737e+03 5.85800e+00 1.73800e+00]


In [8]:
A = G_s
b = d[0]

m0, rnorm = nnls(A, b)
print("Model estimation:", m0, m0.sum())
print("Check:")
print("Density:", round(np.dot(m0, properties_rho_s), 2))
print("vP:", round(np.dot(m0, properties_vP_s), 2))
print("vS:", round(np.dot(m0, properties_vS_s), 2))
print("GR:", round(np.dot(m0, properties_GR_s), 2))
print("PE:", round(np.dot(m0, properties_PE_s), 2))
print("Reality:")
print(d[0])

Model estimation: [0.48840711 0.         0.        ] 0.4884071125992352
Check:
Density: 1293.6
vP: 2809.9
vS: 1990.7
GR: 0.0
PE: 0.81
Reality:
[2.12100e+03 2.75724e+03 1.52737e+03 5.85800e+00 1.73800e+00]


In [9]:
import numpy as np
from scipy.optimize import minimize 
from scipy.optimize import nnls 

#Define problem
A = np.array([[60, 90, 120], 
              [30, 120, 90]])
A = G

b = np.array([67.5, 60])
b = d[0]

#Use nnls to get initial guess
x0, rnorm = nnls(A, b)

#Define minimisation function
def fn(x, A, b):
    return np.linalg.norm(A.dot(x) - b)

#Define constraints and bounds
cons = {'type': 'eq', 'fun': lambda x:  np.sum(x)-1}
bounds = [[0.5, 1.0], [0., 0.5], [0., 0.5], [0., 0.2]]

#Call minimisation subject to these values
minout = minimize(fn, x0, args=(A, b), method="SLSQP", bounds=bounds, constraints=cons)
x = minout.x

print(x, x.sum(), fn(x,A,b))
m0 = x

print("")
print("Model estimation:", m0, m0.sum())
print("Check:")
print("Density:", round(np.dot(m0,properties_rho), 2), "-->", round((np.dot(m0,properties_rho)/d[0][0]-1)*100, 2), "% difference")
print("vP:", round(np.dot(m0,properties_vP), 2), "-->", round((np.dot(m0,properties_vP)/d[0][1]-1)*100, 2), "% difference")
print("vS:", round(np.dot(m0,properties_vS), 2), "-->", round((np.dot(m0,properties_vS)/d[0][2]-1)*100, 2), "% difference")
print("GR:", round(np.dot(m0,properties_GR), 2), "-->", round((np.dot(m0,properties_GR)/d[0][3]-1)*100, 2), "% difference")
print("PE:", round(np.dot(m0,properties_PE), 2), "-->", round((np.dot(m0,properties_PE)/d[0][4]-1)*100, 2), "% difference")
print("Reality:")
print(d[0])

[0.80000054 0.         0.         0.19999985] 1.0000003901133905 2757.2697513127464

Model estimation: [0.80000054 0.         0.         0.19999985] 1.0000003901133905
Check:
Density: 2318.88 --> 9.33 % difference
vP: 4892.38 --> 77.44 % difference
vS: 3260.72 --> 113.49 % difference
GR: 0.0 --> -100.0 % difference
PE: 1.39 --> -19.91 % difference
Reality:
[2.12100e+03 2.75724e+03 1.52737e+03 5.85800e+00 1.73800e+00]


In [10]:
import numpy as np
from scipy.optimize import minimize 
from scipy.optimize import nnls 

#Define problem
A = G_s
b = d[0]

#Use nnls to get initial guess
x0, rnorm = nnls(A, b)

#Define minimisation function
def fn(x, A, b):
    return np.linalg.norm(A.dot(x) - b)

#Define constraints and bounds
cons = {'type': 'eq', 'fun': lambda x:  np.sum(x)-1}
bounds = [[0.5, 1.0], [0., 0.5], [0., 0.5]]

#Call minimisation subject to these values
minout = minimize(fn, x0, args=(A, b), method="SLSQP", bounds=bounds, constraints=cons)
x = minout.x

print(x, x.sum(), fn(x, A, b))
m0 = x

print("")
print("Model estimation:", m0, m0.sum())
print("Check:")
print("Density:", round(np.dot(m0, properties_rho_s), 2), "-->", round((np.dot(m0, properties_rho_s)/d[0][0]-1)*100, 2), "% difference")
print("vP:", round(np.dot(m0, properties_vP_s), 2), "-->", round((np.dot(m0, properties_vP_s)/d[0][1]-1)*100, 2), "% difference")
print("vS:", round(np.dot(m0, properties_vS_s), 2), "-->", round((np.dot(m0, properties_vS_s)/d[0][2]-1)*100, 2), "% difference")
print("GR:", round(np.dot(m0, properties_GR_s), 2), "-->", round((np.dot(m0, properties_GR_s)/d[0][3]-1)*100, 2), "% difference")
print("PE:", round(np.dot(m0, properties_PE_s), 2), "-->", round((np.dot(m0, properties_PE_s)/d[0][4]-1)*100, 2), "% difference")
print("Reality:")
print(d[0])

[1. 0. 0.] 1.0 3968.523348351122

Model estimation: [1. 0. 0.] 1.0
Check:
Density: 2648.6 --> 24.88 % difference
vP: 5753.2 --> 108.66 % difference
vS: 4075.9 --> 166.86 % difference
GR: 0.0 --> -100.0 % difference
PE: 1.65 --> -5.06 % difference
Reality:
[2.12100e+03 2.75724e+03 1.52737e+03 5.85800e+00 1.73800e+00]


In [40]:
step = 0.1

w_Qz = np.linspace(0.5, 1.0, 201)
w_Kfs = np.linspace(0, 0.5, 201)
w_Pl = np.linspace(0, 0.5, 201)
w_H2O = np.linspace(0, 0.2, 41)

#print("Qz:", w_Qz, len(w_Qz))
#print("Kfs:", w_Kfs, len(w_Kfs))
#print("Pl:", w_Pl, len(w_Pl))
#print("H2O:", w_H2O, len(w_H2O))
#print("")

rho_calc = []
rho_indices = []
rho_w = []
rho_sample = d[0][0]
rho_Qz = properties_rho[0]
rho_Kfs = properties_rho[1]
rho_Pl = properties_rho[2]
rho_H2O = properties_rho[3]
phi_values = []

print("Calculation started!")
for i in w_Qz:
    for j in w_Kfs:
        for k in w_Pl:
            for l in w_H2O:
                phi = round((l/rho_H2O)/(i/rho_Qz + j/rho_Kfs + k/rho_Pl + l/rho_H2O), 4)
                value = (1-phi)*(i*rho_Qz + j*rho_Kfs + k*rho_Pl) + phi*rho_H2O
                if 0.9975*rho_sample < value and value <= 1.0025*rho_sample:
                    w = round(i+j+k+l, 2)
                    #print("w =", w)
                    if w == 1.0:
                        #print("phi =", phi, "for", i, j , k, l)
                        rho_calc.append(value)
                        rho_w.append([i, j, k, l])
                        phi_values.append(phi)
                
print("Calculation done!")
rho_calc = np.array(rho_calc)
print("Datapoints:", len(rho_calc))

Calculation started!
Calculation done!
Datapoints: 25599


In [41]:
print("Checking started!")
results_rho = rho_calc[(0.9975*d[0][0] < rho_calc) & (rho_calc <= 1.0025*d[0][0])]
rho_indices = []
for i in results_rho:
    rho_indices.append(np.where(rho_calc==i))
print("Checking done!")

print("Real density:", d[0][0])
print("")

print("Found models:", len(results_rho))
for i in range(len(results_rho)):
    if round(sum(np.around(rho_w[rho_indices[i][0][0]], 4)), 4) == 1.0:
        print("Calculated density:", round(results_rho[i], 1))
        print("Weight amounts:", np.around(rho_w[rho_indices[i][0][0]], 4))
        print("Porosity:", np.around(phi_values[i], 4))
print("Finished!")

Checking started!
Checking done!
Real density: 2160.0

Found models: 25599
Calculated density: 2161.7
Weight amounts: [0.5  0.   0.42 0.08]
Porosity: 0.1867
Calculated density: 2161.6
Weight amounts: [0.5    0.0025 0.4175 0.08  ]
Porosity: 0.1867
Calculated density: 2161.5
Weight amounts: [0.5   0.005 0.415 0.08 ]
Porosity: 0.1867
Calculated density: 2161.4
Weight amounts: [0.5    0.0075 0.4125 0.08  ]
Porosity: 0.1867
Calculated density: 2161.4
Weight amounts: [0.5  0.01 0.41 0.08]
Porosity: 0.1866
Calculated density: 2161.4
Weight amounts: [0.5    0.0125 0.4075 0.08  ]
Porosity: 0.1866
Calculated density: 2161.3
Weight amounts: [0.5   0.015 0.405 0.08 ]
Porosity: 0.1866
Calculated density: 2161.2
Weight amounts: [0.5    0.0175 0.4025 0.08  ]
Porosity: 0.1866
Calculated density: 2161.1
Weight amounts: [0.5  0.02 0.4  0.08]
Porosity: 0.1866
Calculated density: 2161.0
Weight amounts: [0.5    0.0225 0.3975 0.08  ]
Porosity: 0.1866
Calculated density: 2160.9
Weight amounts: [0.5   0.025 0

Porosity: 0.1866
Calculated density: 2160.7
Weight amounts: [0.5075 0.0325 0.38   0.08  ]
Porosity: 0.1866
Calculated density: 2160.6
Weight amounts: [0.5075 0.035  0.3775 0.08  ]
Porosity: 0.1866
Calculated density: 2160.5
Weight amounts: [0.5075 0.0375 0.375  0.08  ]
Porosity: 0.1866
Calculated density: 2160.5
Weight amounts: [0.5075 0.04   0.3725 0.08  ]
Porosity: 0.1866
Calculated density: 2160.4
Weight amounts: [0.5075 0.0425 0.37   0.08  ]
Porosity: 0.1866
Calculated density: 2160.3
Weight amounts: [0.5075 0.045  0.3675 0.08  ]
Porosity: 0.1866
Calculated density: 2160.3
Weight amounts: [0.5075 0.0475 0.365  0.08  ]
Porosity: 0.1865
Calculated density: 2160.2
Weight amounts: [0.5075 0.05   0.3625 0.08  ]
Porosity: 0.1865
Calculated density: 2160.1
Weight amounts: [0.5075 0.0525 0.36   0.08  ]
Porosity: 0.1865
Calculated density: 2160.0
Weight amounts: [0.5075 0.055  0.3575 0.08  ]
Porosity: 0.1865
Calculated density: 2159.9
Weight amounts: [0.5075 0.0575 0.355  0.08  ]
Porosity: 

Weight amounts: [0.5125 0.1225 0.285  0.08  ]
Porosity: 0.1863
Calculated density: 2157.8
Weight amounts: [0.5125 0.125  0.2825 0.08  ]
Porosity: 0.1863
Calculated density: 2157.7
Weight amounts: [0.5125 0.1275 0.28   0.08  ]
Porosity: 0.1863
Calculated density: 2157.6
Weight amounts: [0.5125 0.13   0.2775 0.08  ]
Porosity: 0.1863
Calculated density: 2157.5
Weight amounts: [0.5125 0.1325 0.275  0.08  ]
Porosity: 0.1863
Calculated density: 2157.4
Weight amounts: [0.5125 0.135  0.2725 0.08  ]
Porosity: 0.1863
Calculated density: 2157.3
Weight amounts: [0.5125 0.1375 0.27   0.08  ]
Porosity: 0.1863
Calculated density: 2157.2
Weight amounts: [0.5125 0.14   0.2675 0.08  ]
Porosity: 0.1863
Calculated density: 2157.1
Weight amounts: [0.5125 0.1425 0.265  0.08  ]
Porosity: 0.1863
Calculated density: 2157.0
Weight amounts: [0.5125 0.145  0.2625 0.08  ]
Porosity: 0.1863
Calculated density: 2156.9
Weight amounts: [0.5125 0.1475 0.26   0.08  ]
Porosity: 0.1863
Calculated density: 2157.0
Weight amo

Porosity: 0.1861
Calculated density: 2154.7
Weight amounts: [0.5175 0.2175 0.185  0.08  ]
Porosity: 0.1861
Calculated density: 2154.6
Weight amounts: [0.5175 0.22   0.1825 0.08  ]
Porosity: 0.1861
Calculated density: 2154.7
Weight amounts: [0.5175 0.2225 0.18   0.08  ]
Porosity: 0.186
Calculated density: 2162.0
Weight amounts: [0.52 0.   0.4  0.08]
Porosity: 0.1867
Calculated density: 2161.9
Weight amounts: [0.52   0.0025 0.3975 0.08  ]
Porosity: 0.1867
Calculated density: 2161.8
Weight amounts: [0.52  0.005 0.395 0.08 ]
Porosity: 0.1867
Calculated density: 2161.7
Weight amounts: [0.52   0.0075 0.3925 0.08  ]
Porosity: 0.1867
Calculated density: 2161.6
Weight amounts: [0.52 0.01 0.39 0.08]
Porosity: 0.1867
Calculated density: 2161.5
Weight amounts: [0.52   0.0125 0.3875 0.08  ]
Porosity: 0.1867
Calculated density: 2161.4
Weight amounts: [0.52  0.015 0.385 0.08 ]
Porosity: 0.1867
Calculated density: 2161.5
Weight amounts: [0.52   0.0175 0.3825 0.08  ]
Porosity: 0.1866
Calculated density

Weight amounts: [0.525 0.06  0.335 0.08 ]
Porosity: 0.1865
Calculated density: 2160.0
Weight amounts: [0.525  0.0625 0.3325 0.08  ]
Porosity: 0.1865
Calculated density: 2159.9
Weight amounts: [0.525 0.065 0.33  0.08 ]
Porosity: 0.1865
Calculated density: 2159.9
Weight amounts: [0.525  0.0675 0.3275 0.08  ]
Porosity: 0.1865
Calculated density: 2159.8
Weight amounts: [0.525 0.07  0.325 0.08 ]
Porosity: 0.1865
Calculated density: 2159.7
Weight amounts: [0.525  0.0725 0.3225 0.08  ]
Porosity: 0.1865
Calculated density: 2159.6
Weight amounts: [0.525 0.075 0.32  0.08 ]
Porosity: 0.1865
Calculated density: 2159.5
Weight amounts: [0.525  0.0775 0.3175 0.08  ]
Porosity: 0.1865
Calculated density: 2159.4
Weight amounts: [0.525 0.08  0.315 0.08 ]
Porosity: 0.1865
Calculated density: 2159.3
Weight amounts: [0.525  0.0825 0.3125 0.08  ]
Porosity: 0.1865
Calculated density: 2159.2
Weight amounts: [0.525 0.085 0.31  0.08 ]
Porosity: 0.1865
Calculated density: 2159.2
Weight amounts: [0.525  0.0875 0.3

...

Calculated density: 2161.1
Weight amounts: [0.78 0.14 0.   0.08]
Porosity: 0.1866
Calculated density: 2165.4
Weight amounts: [0.7825 0.0125 0.125  0.08  ]
Porosity: 0.187
Calculated density: 2165.3
Weight amounts: [0.7825 0.015  0.1225 0.08  ]
Porosity: 0.187
Calculated density: 2165.2
Weight amounts: [0.7825 0.0175 0.12   0.08  ]
Porosity: 0.187
Calculated density: 2165.1
Weight amounts: [0.7825 0.02   0.1175 0.08  ]
Porosity: 0.187
Calculated density: 2165.0
Weight amounts: [0.7825 0.0225 0.115  0.08  ]
Porosity: 0.187
Calculated density: 2164.9
Weight amounts: [0.7825 0.025  0.1125 0.08  ]
Porosity: 0.187
Calculated density: 2164.9
Weight amounts: [0.7825 0.0275 0.11   0.08  ]
Porosity: 0.1869
Calculated density: 2164.8
Weight amounts: [0.7825 0.03   0.1075 0.08  ]
Porosity: 0.1869
Calculated density: 2164.8
Weight amounts: [0.7825 0.0325 0.105  0.08  ]
Porosity: 0.1869
Calculated density: 2164.7
Weight amounts: [0.7825 0.035  0.1025 0.08  ]
Porosity: 0.1869
Calculated density: 2164

Calculated density: 2164.9
Weight amounts: [0.79   0.0275 0.1025 0.08  ]
Porosity: 0.187
Calculated density: 2165.0
Weight amounts: [0.79 0.03 0.1  0.08]
Porosity: 0.1869
Calculated density: 2164.9
Weight amounts: [0.79   0.0325 0.0975 0.08  ]
Porosity: 0.1869
Calculated density: 2164.8
Weight amounts: [0.79  0.035 0.095 0.08 ]
Porosity: 0.1869
Calculated density: 2164.7
Weight amounts: [0.79   0.0375 0.0925 0.08  ]
Porosity: 0.1869
Calculated density: 2164.6
Weight amounts: [0.79 0.04 0.09 0.08]
Porosity: 0.1869
Calculated density: 2164.5
Weight amounts: [0.79   0.0425 0.0875 0.08  ]
Porosity: 0.1869
Calculated density: 2164.4
Weight amounts: [0.79  0.045 0.085 0.08 ]
Porosity: 0.1869
Calculated density: 2164.3
Weight amounts: [0.79   0.0475 0.0825 0.08  ]
Porosity: 0.1869
Calculated density: 2164.2
Weight amounts: [0.79 0.05 0.08 0.08]
Porosity: 0.1869
Calculated density: 2164.1
Weight amounts: [0.79   0.0525 0.0775 0.08  ]
Porosity: 0.1869
Calculated density: 2164.0
Weight amounts: 

Calculated density: 2163.6
Weight amounts: [0.8   0.075 0.045 0.08 ]
Porosity: 0.1868
Calculated density: 2163.5
Weight amounts: [0.8    0.0775 0.0425 0.08  ]
Porosity: 0.1868
Calculated density: 2163.4
Weight amounts: [0.8  0.08 0.04 0.08]
Porosity: 0.1868
Calculated density: 2163.3
Weight amounts: [0.8    0.0825 0.0375 0.08  ]
Porosity: 0.1868
Calculated density: 2163.2
Weight amounts: [0.8   0.085 0.035 0.08 ]
Porosity: 0.1868
Calculated density: 2163.1
Weight amounts: [0.8    0.0875 0.0325 0.08  ]
Porosity: 0.1868
Calculated density: 2163.0
Weight amounts: [0.8  0.09 0.03 0.08]
Porosity: 0.1868
Calculated density: 2162.9
Weight amounts: [0.8    0.0925 0.0275 0.08  ]
Porosity: 0.1868
Calculated density: 2162.8
Weight amounts: [0.8   0.095 0.025 0.08 ]
Porosity: 0.1868
Calculated density: 2162.8
Weight amounts: [0.8    0.0975 0.0225 0.08  ]
Porosity: 0.1868
Calculated density: 2162.7
Weight amounts: [0.8  0.1  0.02 0.08]
Porosity: 0.1868
Calculated density: 2162.7
Weight amounts: [0.

Weight amounts: [0.815  0.0275 0.0775 0.08  ]
Porosity: 0.187
Calculated density: 2165.2
Weight amounts: [0.815 0.03  0.075 0.08 ]
Porosity: 0.187
Calculated density: 2165.1
Weight amounts: [0.815  0.0325 0.0725 0.08  ]
Porosity: 0.187
Calculated density: 2165.0
Weight amounts: [0.815 0.035 0.07  0.08 ]
Porosity: 0.187
Calculated density: 2164.9
Weight amounts: [0.815  0.0375 0.0675 0.08  ]
Porosity: 0.187
Calculated density: 2165.0
Weight amounts: [0.815 0.04  0.065 0.08 ]
Porosity: 0.1869
Calculated density: 2164.9
Weight amounts: [0.815  0.0425 0.0625 0.08  ]
Porosity: 0.1869
Calculated density: 2164.8
Weight amounts: [0.815 0.045 0.06  0.08 ]
Porosity: 0.1869
Calculated density: 2164.7
Weight amounts: [0.815  0.0475 0.0575 0.08  ]
Porosity: 0.1869
Calculated density: 2164.6
Weight amounts: [0.815 0.05  0.055 0.08 ]
Porosity: 0.1869
Calculated density: 2164.5
Weight amounts: [0.815  0.0525 0.0525 0.08  ]
Porosity: 0.1869
Calculated density: 2164.4
Weight amounts: [0.815 0.055 0.05  

Calculated density: 2164.5
Weight amounts: [0.83 0.06 0.03 0.08]
Porosity: 0.1869
Calculated density: 2164.4
Weight amounts: [0.83   0.0625 0.0275 0.08  ]
Porosity: 0.1869
Calculated density: 2164.3
Weight amounts: [0.83  0.065 0.025 0.08 ]
Porosity: 0.1869
Calculated density: 2164.2
Weight amounts: [0.83   0.0675 0.0225 0.08  ]
Porosity: 0.1869
Calculated density: 2164.1
Weight amounts: [0.83 0.07 0.02 0.08]
Porosity: 0.1869
Calculated density: 2164.0
Weight amounts: [0.83   0.0725 0.0175 0.08  ]
Porosity: 0.1869
Calculated density: 2163.9
Weight amounts: [0.83  0.075 0.015 0.08 ]
Porosity: 0.1869
Calculated density: 2163.8
Weight amounts: [0.83   0.0775 0.0125 0.08  ]
Porosity: 0.1869
Calculated density: 2163.9
Weight amounts: [0.83 0.08 0.01 0.08]
Porosity: 0.1868
Calculated density: 2163.8
Weight amounts: [0.83   0.0825 0.0075 0.08  ]
Porosity: 0.1868
Calculated density: 2163.7
Weight amounts: [0.83  0.085 0.005 0.08 ]
Porosity: 0.1868
Calculated density: 2163.6
Weight amounts: [0.

In [51]:
w_Qz = np.linspace(0.5, 1.0, 201)
w_Kfs = np.linspace(0, 0.5, 201)
w_Pl = np.linspace(0, 0.5, 201)
w_H2O = np.linspace(0, 0.2, 41)

print("Calculation started!")
for i in w_Qz:
    for j in w_Kfs:
        for k in w_Pl:
            for l in w_H2O:
                phi = round((l/rho_H2O)/(i/rho_Qz + j/rho_Kfs + k/rho_Pl + l/rho_H2O), 4)
                w = round(i+j+k+l, 2)
                if w == 1.0:
                    value = round((1-phi)*(i*rho_Qz + j*rho_Kfs + k*rho_Pl) + phi*rho_H2O, 1)
                    if 2100 < value < 2200:
                        print("rho =", value)
                        print("phi =", phi)
                        print("for ", np.around([i, j, k, l], 4))
print("Calculation done!")

Calculation started!
rho = 2100.2
phi = 0.2079
for  [0.5   0.    0.405 0.09 ]
rho = 2105.9
phi = 0.2075
for  [0.5    0.     0.4075 0.09  ]
rho = 2124.7
phi = 0.1978
for  [0.5   0.    0.41  0.085]
rho = 2111.8
phi = 0.207
for  [0.5  0.   0.41 0.09]
rho = 2130.6
phi = 0.1974
for  [0.5    0.     0.4125 0.085 ]
rho = 2117.6
phi = 0.2066
for  [0.5    0.     0.4125 0.09  ]
rho = 2149.9
phi = 0.1875
for  [0.5   0.    0.415 0.08 ]
rho = 2136.6
phi = 0.1969
for  [0.5   0.    0.415 0.085]
rho = 2155.8
phi = 0.1871
for  [0.5    0.     0.4175 0.08  ]
rho = 2142.4
phi = 0.1965
for  [0.5    0.     0.4175 0.085 ]
rho = 2161.7
phi = 0.1867
for  [0.5  0.   0.42 0.08]
rho = 2148.3
phi = 0.1961
for  [0.5   0.    0.42  0.085]
rho = 2181.4
phi = 0.1767
for  [0.5    0.     0.4225 0.075 ]
rho = 2167.6
phi = 0.1863
for  [0.5    0.     0.4225 0.08  ]
rho = 2187.4
phi = 0.1763
for  [0.5   0.    0.425 0.075]
rho = 2193.4
phi = 0.1759
for  [0.5    0.     0.4275 0.075 ]
rho = 2199.4
phi = 0.1755
for  [0.5   0.    

phi = 0.1758
for  [0.5    0.02   0.4075 0.075 ]
rho = 2105.3
phi = 0.2074
for  [0.5    0.0225 0.385  0.09  ]
rho = 2111.2
phi = 0.2069
for  [0.5    0.0225 0.3875 0.09  ]
rho = 2129.9
phi = 0.1973
for  [0.5    0.0225 0.39   0.085 ]
rho = 2116.9
phi = 0.2065
for  [0.5    0.0225 0.39   0.09  ]
rho = 2149.2
phi = 0.1874
for  [0.5    0.0225 0.3925 0.08  ]
rho = 2135.9
phi = 0.1968
for  [0.5    0.0225 0.3925 0.085 ]
rho = 2155.1
phi = 0.187
for  [0.5    0.0225 0.395  0.08  ]
rho = 2141.7
phi = 0.1964
for  [0.5    0.0225 0.395  0.085 ]
rho = 2161.0
phi = 0.1866
for  [0.5    0.0225 0.3975 0.08  ]
rho = 2147.6
phi = 0.196
for  [0.5    0.0225 0.3975 0.085 ]
rho = 2180.7
phi = 0.1766
for  [0.5    0.0225 0.4    0.075 ]
rho = 2166.9
phi = 0.1862
for  [0.5    0.0225 0.4    0.08  ]
rho = 2186.7
phi = 0.1762
for  [0.5    0.0225 0.4025 0.075 ]
rho = 2192.6
phi = 0.1758
for  [0.5    0.0225 0.405  0.075 ]
rho = 2198.6
phi = 0.1754
for  [0.5    0.0225 0.4075 0.075 ]
rho = 2105.2
phi = 0.2074
for  [0.5    

rho = 2104.4
phi = 0.2073
for  [0.5    0.05   0.3575 0.09  ]
rho = 2123.2
phi = 0.1976
for  [0.5   0.05  0.36  0.085]
rho = 2110.2
phi = 0.2069
for  [0.5  0.05 0.36 0.09]
rho = 2129.0
phi = 0.1972
for  [0.5    0.05   0.3625 0.085 ]
rho = 2116.1
phi = 0.2064
for  [0.5    0.05   0.3625 0.09  ]
rho = 2148.1
phi = 0.1874
for  [0.5   0.05  0.365 0.08 ]
rho = 2134.9
phi = 0.1968
for  [0.5   0.05  0.365 0.085]
rho = 2154.2
phi = 0.1869
for  [0.5    0.05   0.3675 0.08  ]
rho = 2140.8
phi = 0.1963
for  [0.5    0.05   0.3675 0.085 ]
rho = 2173.8
phi = 0.1769
for  [0.5   0.05  0.37  0.075]
rho = 2160.1
phi = 0.1865
for  [0.5  0.05 0.37 0.08]
rho = 2179.8
phi = 0.1765
for  [0.5    0.05   0.3725 0.075 ]
rho = 2166.0
phi = 0.1861
for  [0.5    0.05   0.3725 0.08  ]
rho = 2199.7
phi = 0.1664
for  [0.5   0.05  0.375 0.07 ]
rho = 2185.8
phi = 0.1761
for  [0.5   0.05  0.375 0.075]
rho = 2191.8
phi = 0.1757
for  [0.5    0.05   0.3775 0.075 ]
rho = 2104.3
phi = 0.2073
for  [0.5    0.0525 0.355  0.09  ]
rho

rho = 2103.7
phi = 0.2072
for  [0.5    0.0725 0.335  0.09  ]
rho = 2122.3
phi = 0.1976
for  [0.5    0.0725 0.3375 0.085 ]
rho = 2109.5
phi = 0.2068
for  [0.5    0.0725 0.3375 0.09  ]
rho = 2128.3
phi = 0.1971
for  [0.5    0.0725 0.34   0.085 ]
rho = 2115.4
phi = 0.2063
for  [0.5    0.0725 0.34   0.09  ]
rho = 2147.4
phi = 0.1873
for  [0.5    0.0725 0.3425 0.08  ]
rho = 2134.2
phi = 0.1967
for  [0.5    0.0725 0.3425 0.085 ]
rho = 2153.3
phi = 0.1869
for  [0.5    0.0725 0.345  0.08  ]
rho = 2140.0
phi = 0.1963
for  [0.5    0.0725 0.345  0.085 ]
rho = 2172.9
phi = 0.1769
for  [0.5    0.0725 0.3475 0.075 ]
rho = 2159.3
phi = 0.1865
for  [0.5    0.0725 0.3475 0.08  ]
rho = 2178.9
phi = 0.1765
for  [0.5    0.0725 0.35   0.075 ]
rho = 2165.3
phi = 0.186
for  [0.5    0.0725 0.35   0.08  ]
rho = 2199.0
phi = 0.1663
for  [0.5    0.0725 0.3525 0.07  ]
rho = 2184.9
phi = 0.1761
for  [0.5    0.0725 0.3525 0.075 ]
rho = 2190.9
phi = 0.1757
for  [0.5    0.0725 0.355  0.075 ]
rho = 2196.9
phi = 0.1753

rho = 2102.8
phi = 0.2071
for  [0.5    0.1    0.3075 0.09  ]
rho = 2108.6
phi = 0.2067
for  [0.5  0.1  0.31 0.09]
rho = 2127.5
phi = 0.197
for  [0.5    0.1    0.3125 0.085 ]
rho = 2114.5
phi = 0.2062
for  [0.5    0.1    0.3125 0.09  ]
rho = 2146.6
phi = 0.1872
for  [0.5   0.1   0.315 0.08 ]
rho = 2133.3
phi = 0.1966
for  [0.5   0.1   0.315 0.085]
rho = 2152.5
phi = 0.1868
for  [0.5    0.1    0.3175 0.08  ]
rho = 2139.1
phi = 0.1962
for  [0.5    0.1    0.3175 0.085 ]
rho = 2158.4
phi = 0.1864
for  [0.5  0.1  0.32 0.08]
rho = 2145.1
phi = 0.1957
for  [0.5   0.1   0.32  0.085]
rho = 2178.0
phi = 0.1764
for  [0.5    0.1    0.3225 0.075 ]
rho = 2164.3
phi = 0.186
for  [0.5    0.1    0.3225 0.08  ]
rho = 2198.1
phi = 0.1662
for  [0.5   0.1   0.325 0.07 ]
rho = 2184.0
phi = 0.176
for  [0.5   0.1   0.325 0.075]
rho = 2190.0
phi = 0.1756
for  [0.5    0.1    0.3275 0.075 ]
rho = 2196.0
phi = 0.1752
for  [0.5   0.1   0.33  0.075]
rho = 2102.8
phi = 0.2071
for  [0.5    0.1025 0.305  0.09  ]
rho = 

...

rho = 2102.3
phi = 0.2061
for  [0.5125 0.3025 0.095  0.09  ]
rho = 2121.0
phi = 0.1964
for  [0.5125 0.3025 0.0975 0.085 ]
rho = 2108.2
phi = 0.2056
for  [0.5125 0.3025 0.0975 0.09  ]
rho = 2126.9
phi = 0.196
for  [0.5125 0.3025 0.1    0.085 ]
rho = 2114.0
phi = 0.2052
for  [0.5125 0.3025 0.1    0.09  ]
rho = 2145.9
phi = 0.1862
for  [0.5125 0.3025 0.1025 0.08  ]
rho = 2132.7
phi = 0.1956
for  [0.5125 0.3025 0.1025 0.085 ]
rho = 2151.9
phi = 0.1858
for  [0.5125 0.3025 0.105  0.08  ]
rho = 2138.7
phi = 0.1951
for  [0.5125 0.3025 0.105  0.085 ]
rho = 2171.4
phi = 0.1758
for  [0.5125 0.3025 0.1075 0.075 ]
rho = 2157.8
phi = 0.1854
for  [0.5125 0.3025 0.1075 0.08  ]
rho = 2177.3
phi = 0.1755
for  [0.5125 0.3025 0.11   0.075 ]
rho = 2163.7
phi = 0.185
for  [0.5125 0.3025 0.11   0.08  ]
rho = 2197.3
phi = 0.1653
for  [0.5125 0.3025 0.1125 0.07  ]
rho = 2183.3
phi = 0.1751
for  [0.5125 0.3025 0.1125 0.075 ]
rho = 2189.3
phi = 0.1747
for  [0.5125 0.3025 0.115  0.075 ]
rho = 2102.2
phi = 0.2061


for  [0.5125 0.325  0.085  0.08  ]
rho = 2176.6
phi = 0.1754
for  [0.5125 0.325  0.0875 0.075 ]
rho = 2163.0
phi = 0.1849
for  [0.5125 0.325  0.0875 0.08  ]
rho = 2196.5
phi = 0.1653
for  [0.5125 0.325  0.09   0.07  ]
rho = 2182.6
phi = 0.175
for  [0.5125 0.325  0.09   0.075 ]
rho = 2188.6
phi = 0.1746
for  [0.5125 0.325  0.0925 0.075 ]
rho = 2101.5
phi = 0.206
for  [0.5125 0.3275 0.07   0.09  ]
rho = 2120.1
phi = 0.1964
for  [0.5125 0.3275 0.0725 0.085 ]
rho = 2107.4
phi = 0.2055
for  [0.5125 0.3275 0.0725 0.09  ]
rho = 2126.1
phi = 0.1959
for  [0.5125 0.3275 0.075  0.085 ]
rho = 2113.2
phi = 0.2051
for  [0.5125 0.3275 0.075  0.09  ]
rho = 2145.2
phi = 0.1861
for  [0.5125 0.3275 0.0775 0.08  ]
rho = 2131.9
phi = 0.1955
for  [0.5125 0.3275 0.0775 0.085 ]
rho = 2151.1
phi = 0.1857
for  [0.5125 0.3275 0.08   0.08  ]
rho = 2137.8
phi = 0.1951
for  [0.5125 0.3275 0.08   0.085 ]
rho = 2170.5
phi = 0.1758
for  [0.5125 0.3275 0.0825 0.075 ]
rho = 2157.0
phi = 0.1853
for  [0.5125 0.3275 0.0825

phi = 0.1963
for  [0.5125 0.3475 0.0525 0.085 ]
rho = 2106.7
phi = 0.2055
for  [0.5125 0.3475 0.0525 0.09  ]
rho = 2138.5
phi = 0.1865
for  [0.5125 0.3475 0.055  0.08  ]
rho = 2125.3
phi = 0.1959
for  [0.5125 0.3475 0.055  0.085 ]
rho = 2144.4
phi = 0.1861
for  [0.5125 0.3475 0.0575 0.08  ]
rho = 2131.3
phi = 0.1954
for  [0.5125 0.3475 0.0575 0.085 ]
rho = 2150.3
phi = 0.1857
for  [0.5125 0.3475 0.06   0.08  ]
rho = 2137.2
phi = 0.195
for  [0.5125 0.3475 0.06   0.085 ]
rho = 2169.9
phi = 0.1757
for  [0.5125 0.3475 0.0625 0.075 ]
rho = 2156.2
phi = 0.1853
for  [0.5125 0.3475 0.0625 0.08  ]
rho = 2189.7
phi = 0.1656
for  [0.5125 0.3475 0.065  0.07  ]
rho = 2175.9
phi = 0.1753
for  [0.5125 0.3475 0.065  0.075 ]
rho = 2195.7
phi = 0.1652
for  [0.5125 0.3475 0.0675 0.07  ]
rho = 2181.8
phi = 0.1749
for  [0.5125 0.3475 0.0675 0.075 ]
rho = 2187.7
phi = 0.1746
for  [0.5125 0.3475 0.07   0.075 ]
rho = 2113.6
phi = 0.1967
for  [0.5125 0.35   0.0475 0.085 ]
rho = 2100.8
phi = 0.2059
for  [0.5125

phi = 0.1753
for  [0.5125 0.3675 0.045  0.075 ]
rho = 2161.6
phi = 0.1848
for  [0.5125 0.3675 0.045  0.08  ]
rho = 2195.0
phi = 0.1652
for  [0.5125 0.3675 0.0475 0.07  ]
rho = 2181.1
phi = 0.1749
for  [0.5125 0.3675 0.0475 0.075 ]
rho = 2187.1
phi = 0.1745
for  [0.5125 0.3675 0.05   0.075 ]
rho = 2100.2
phi = 0.2058
for  [0.5125 0.37   0.0275 0.09  ]
rho = 2118.8
phi = 0.1962
for  [0.5125 0.37   0.03   0.085 ]
rho = 2106.0
phi = 0.2054
for  [0.5125 0.37   0.03   0.09  ]
rho = 2124.7
phi = 0.1958
for  [0.5125 0.37   0.0325 0.085 ]
rho = 2111.8
phi = 0.205
for  [0.5125 0.37   0.0325 0.09  ]
rho = 2143.7
phi = 0.186
for  [0.5125 0.37   0.035  0.08  ]
rho = 2130.5
phi = 0.1954
for  [0.5125 0.37   0.035  0.085 ]
rho = 2149.6
phi = 0.1856
for  [0.5125 0.37   0.0375 0.08  ]
rho = 2136.5
phi = 0.1949
for  [0.5125 0.37   0.0375 0.085 ]
rho = 2169.0
phi = 0.1757
for  [0.5125 0.37   0.04   0.075 ]
rho = 2155.5
phi = 0.1852
for  [0.5125 0.37   0.04   0.08  ]
rho = 2175.0
phi = 0.1753
for  [0.5125 

KeyboardInterrupt: 